In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import os

pipe = pipeline("text-classification", model="rifkat/uz_kr_lang-detection")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [16]:
a = pipe('Salom hammaga')

In [19]:
a[0]['label']

'uz'

In [8]:
TOKEN = '6244200952:AAHyQ22ZiiXqIcfgNV8lAd_R2OQTEjo0Ubc'
CHAT_ID = '111312651'

In [9]:
NEWS_URL = "https://portal.piima.uz/news?page={page}" #1250

LAST = 277

In [2]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import pandas as pd
import requests
import time
from tqdm.contrib.telegram import tqdm

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from bs4 import BeautifulSoup

url = 'https://portal.piima.uz/news?page=277'
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, "lxml")
quotes = soup.find('div', class_='row w-100').find_all('a')

for i in quotes:
    print(i['href'])

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'portal.piima.uz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://portal.piima.uz/news/nukus-shahrida-millatimizni-jahonga-tanitgan-buyuk-ajdodlarmizga-munosib-orinbosarlarni-tarbiyalashga-moljallangan-prezident-maktabi-ochildi
https://portal.piima.uz/news/katta-umidlar-ilk-natijalarini-bermoqda
https://portal.piima.uz/news/prezidentskie-skoly-otkrylis-v-xive-namangane-i-nukuse
https://portal.piima.uz/news/xorazm-vohasidan-buyuk-ajdodlarimiz-vorislari-yetishib-chiqadi
https://portal.piima.uz/news/namanganda-prezident-maktabi-ochildi
https://portal.piima.uz/news/rangli-va-oq-qora-dunyo-rasmlar-va-fotosuratlar-korgazmasi
https://portal.piima.uz/news/sarvar-babaxodzaev-vozglavil-agentstvo-po-razvitiyu-prezidentskix-i-specskol


In [13]:
links = []
async def main():
    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(verify_ssl=False)) as session:
        for page in tqdm(range(1, LAST+1), token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(NEWS_URL.format(page=page)) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        selected_links = soup.find('div', class_='row w-100').find_all('a')
                        links.extend([link['href'] for link in selected_links])
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(e, "page_num", page)
await main()

/tmp/ipykernel_715/1699348387.py:3: DeprecationWarning: verify_ssl is deprecated, use ssl=False instead
  async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(verify_ssl=False)) as session:
100%|█████████████████████████████████████████████████████████████████████████████████| 277/277 [01:42<00:00,  2.70it/s]


In [20]:
results = []
async def main():
    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(verify_ssl=False)) as session:
        for link in tqdm(links, token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(link) as resp:
                    if resp.status == 200:
                        # await asyncio.sleep(1)
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        
                        content = soup.find('div', class_="container py-5").find_all("p")
                        p_list = [p.text for p in content]
                        # if content is None:
                        #     content = soup.find('div', class_="news-content tiny-mc mt-4")

                        new_text = [p for p in p_list if pipe(p)[0]['label'] == "uz"]
                        new_text = "".join(new_text)
                                
                        title = soup.find('h1', class_="hero_box-title").text
                        results.append({
                            "title": title.strip(),
                            "content": new_text
                        })
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(e, link)
await main()

/tmp/ipykernel_715/679143861.py:3: DeprecationWarning: verify_ssl is deprecated, use ssl=False instead
  async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(verify_ssl=False)) as session:
  9%|███████                                                                         | 194/2215 [01:58<49:18,  1.46s/it]

Server disconnected https://portal.piima.uz/news/manim


 13%|██████████▏                                                                     | 283/2215 [02:56<35:50,  1.11s/it]

Server disconnected https://portal.piima.uz/news/yoshlarni-vatanparvarlik-ruhida-tarbiyalash-ustuvor-vazifa


 14%|███████████▌                                                                    | 319/2215 [03:16<17:54,  1.76it/s]

...

 97%|████████████████████████████████████████████████████████████████████████████▉  | 2156/2215 [23:15<00:46,  1.26it/s]

The expanded size of the tensor (1413) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1413].  Tensor sizes: [1, 514] https://portal.piima.uz/news/zamonaviy-olimlar-ajdodlarga-munosib-avlodlar-yetishtirish-burchimizdir


 99%|█████████████████████████████████████████████████████████████████████████████▊ | 2183/2215 [23:40<00:22,  1.43it/s]

The expanded size of the tensor (533) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 533].  Tensor sizes: [1, 514] https://portal.piima.uz/news/prezident-ijod-va-ixtisoslashtirilgan-maktablarda-5-ta-tashabbus-keng-joriy-etilmoqda


100%|███████████████████████████████████████████████████████████████████████████████| 2215/2215 [24:05<00:00,  1.53it/s]


In [21]:
pd.DataFrame(results).to_pickle('piima.pickle')

In [28]:
pd.read_pickle('piima.pickle')['content'][0]

"#bizning_shunday_oʻquvchilarimiz_bor\xa0⚡️Agentligimiz tizimidagi Mirzo Ulug‘bek nomidagi ixtisoslashtirilgan maktabining 11-sinf o’quvchisi, O‘zbekiston terma jamoasi a'zosi Javohir Sultonov 53-Xalqaro fizika olimpiadasida (IPhO) bronza medalini qo‘lga kiritdi.\xa0Ma’lumot uchun, joriy yilning 10-17-iyul kunlari Yaponiyaning Tokio \xa0shahrida bo‘lib o‘tgan mazkur olimpiadada 91 ta mamlakatdan jami 400 nafardan ortiq o‘quvchi ishtirok etdi.\xa0Ushbu medal Xalqaro fizika olimpiadasida(IPHO) qo’lga kiritilgan O’zbekiston tarixidagi 3-bronza medali hisoblanadi.\xa0❗️Terma jamoamiz a’zolari mazkur nufuzli musobaqada 2015-yildan buyon ishtirok etib kelishmoqda va hozirga qadar jami 3 ta bronza medalni (2018, 2021, 2023) qo‘lga kiritishgan.\xa0Javohirni g‘alaba bilan samimiy tabriklab, kelgusidagi o‘qish va ishlarida omad tilaymiz!"